### Download Yolo repository
git clone https://github.com/WongKinYiu/yolov7.git

### Get weight ans save it tp yolov7 directory

wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6e.pt

###  Adapt cfg file

- Create copy of yolov7/cfg/training/yolov7.yaml
- Set number of classes

### Create a file name mask.yaml in yolov7/data/ and put the following configurations:

train: ./train
val: ./val
test: ./test


nc: 3
names: ['with_mask', 'without_mask', 'mask_weared_incorrect']

### Data preparation

- Convert Pascal Voc labels to Yolo format
- Split data into subfolders

In [9]:
# package import

import numpy as np
import pandas as pd
import seaborn as sns
import os
import shutil
import xml.etree.ElementTree as ET
import glob

import json

In [3]:
# Function that convert XML bounding box(xmin, ymin, xmax, ymax)
# https://towardsdatascience.com/convert-pascal-voc-xml-to-yolo-for-object-detection-f969811ccba5
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):
    # x_center y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


In [7]:
# Create folder
def create_folder(path):
    if not os.path.isdir(path):
        os.makedirs(path)

In [21]:
create_folder('yolov7/train/images')
create_folder('yolov7/train/labels')
create_folder('yolov7/test/images')
create_folder('yolov7/test/labels')
create_folder('yolov7/val/images')
create_folder('yolov7/val/labels')

In [ ]:
# get all files
from sklearn import tree


im_folder = 'images'
_, _, files = next(os.walk(im_folder))
pos = 0
for f in files:
    source_img = os.path.join(im_folder, f)
    if pos < 700:
        dest_folder = 'yolov7/train/'
    elif (pos>= 700 and pos< 800):
        dest_folder = 'yolov7/val/'
    else:
        dest_folder = 'yolov7/test/'
    destination_img = os.path.join(dest_folder,'images', f)
    shutil.copy(source_img, destination_img)

    # Check for corresponding labels
    label_file_basename = os.path.splitext(f)[0]
    label_source_file = f"{label_file_basename}.xml"
    
    label_dest_file = f"{label_file_basename}.txt"

    label_source_path = os.path.join('annotations', label_source_file)
    label_dest_path = os.path.join(dest_folder,'labels', label_dest_file)
    
     # if file exists, copy it to target folder
    if os.path.exists(label_source_path):
       # parse the content of the xml file
       tree = ET.parse(label_source_path)
       root = tree.getroot()
       width = int(root.find("size").find("width").text)
       height = int(root.find("size").find("height").text)
       classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']
       result = []

       for obj in root.findall('object'):
           label = obj.find("name").text
           # check for new classes and append to list
           index = classes.index(label)
           pil_bbox = [int(x.text) for x in obj.find("bndbox")]
           yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
           # convert data to string
           bbox_string = " ".join([str(x) for x in yolo_bbox])
           result.append(f"{index} {bbox_string}")
           if result:
               # generate a YOLO format text file for each xml file
               with open(label_dest_path, "w" , encoding="utf-8") as f:
                   f.write("\n".join(result))

    pos += 1
                

### Training
python train.py --weights yolov7-e6e.pt --data data/masks.yaml --workers 4 --batch-size 4 --img 416 --cfg cfg/training/yolov7-masks.yaml --name yolov7 --epochs 5

### Detection

python detect.py --weights runs/train/yolov7/weights/best.pt --conf 0.4 --img-size 640 --source ./test/images/maksssksksss849.png

# Detection with YOLOv8

Can be used in command line directly as:
yolo predict model=yolov8n.pt source='https://ultralytics.com/images/bus.jpg'

or python program:

In [2]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # load a pretrained model

In [3]:
result = model("images/maksssksksss0.png")


image 1/1 /Users/elvist/PycharmProjects/CV Projects/Mask_data/images/maksssksksss0.png: 480x640 4 persons, 2 handbags, 1 cell phone, 73.8ms
Speed: 2.4ms preprocess, 73.8ms inference, 9.8ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
!yolo predict model=yolov8n.pt source='images/maksssksksss0.png' conf=0.3

In [10]:
create_folder('yolov8/train_custom/train/')
create_folder('yolov8/train_custom/test/')
create_folder('yolov8/train_custom/val/')

In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8/train_custom/yolov8n.pt")  # load a pretrained model (recommended for training)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

In [4]:
# Train model
model.train(data="/Users/elvist/PycharmProjects/CV Projects/Mask_detection/yolov8/train_custom/masks.yaml", epochs=100, imgsz=512, batch=4, verbose=True, device='cpu', workers=8)
metrics = model.val()  # evaluate model performance on the validation set
results = model("yolov8/train_custom/test/images/maksssksksss10.png")
success = model.export(format="onnx")  # export the model to ONNX format

New https://pypi.org/project/ultralytics/8.0.105 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.97 🚀 Python-3.10.9 torch-2.0.0 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8/train_custom/yolov8n.pt, data=/Users/elvist/PycharmProjects/CV Projects/Mask_detection/yolov8/train_custom/masks.yaml, epochs=100, patience=50, batch=4, imgsz=512, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [5]:
from ultralytics import YOLO
model = YOLO("runs/detect/train/weights/best.pt")

In [6]:
for result in results:
    print(result.boxes)

WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.yolo.engine.results.Boxes object with attributes:

boxes: tensor([[ 97.3347, 265.5527, 193.2753, 386.2927,   0.9226,   0.0000]])
cls: tensor([0.])
conf: tensor([0.9226])
data: tensor([[ 97.3347, 265.5527, 193.2753, 386.2927,   0.9226,   0.0000]])
id: None
is_track: False
orig_shape: tensor([400, 301])
shape: torch.Size([1, 6])
xywh: tensor([[145.3050, 325.9227,  95.9406, 120.7400]])
xywhn: tensor([[0.4827, 0.8148, 0.3187, 0.3018]])
xyxy: tensor([[ 97.3347, 265.5527, 193.2753, 386.2927]])
xyxyn: tensor([[0.3234, 0.6639, 0.6421, 0.9657]])


In [53]:
results = model("yolov8/train_custom/test/images/maksssksksss690.png")


image 1/1 /Users/elvist/PycharmProjects/CV Projects/Mask_data/yolov8/train_custom/test/images/maksssksksss690.png: 320x512 3 with_masks, 70.5ms
Speed: 1.2ms preprocess, 70.5ms inference, 1.7ms postprocess per image at shape (1, 3, 512, 512)


In [ ]:
import cv2
from google.colab.patches import cv2_imshow # on colab only
res = results[0].plot()
#cv2.imshow("result", res)
cv2_imshow(res)
#cv2.waitKey(0)
#cv2.destroyAllWindows()